In [18]:
import torch
import numpy as np

#### 1灵活的初始化方法
支持list或ndarray输入，默认继承原始的数据类型，并且支持指定数据类型。

In [23]:
# Initializing and basic operations
ccc= [[1, -1], [1, -1]]
a = torch.tensor([[1, -1], [1, -1]])
b = torch.tensor(np.array([[1, 2, 3], [4, 5, 6]]))
c = torch.tensor(np.array([[1, 2, 3], [4, 5, 6]], dtype=np.int64))
d = torch.tensor([[1, -1], [1, -1]], dtype=torch.int32)
print(a)
print(a.dtype)
print('id of a is',id(a),end='\n\n')

print(b)
print(b.dtype)
print('id of b is',id(b),end='\n\n')

print(c)
print(c.dtype)
print('id of c is',id(c),end='\n\n')

print(d)
print(d.dtype)
print('id of d is',id(d))

tensor([[ 1, -1],
        [ 1, -1]])
torch.int64
id of a is 2131336238800

tensor([[1, 2, 3],
        [4, 5, 6]], dtype=torch.int32)
torch.int32
id of b is 2131339419136

tensor([[1, 2, 3],
        [4, 5, 6]])
torch.int64
id of c is 2131339418976

tensor([[ 1, -1],
        [ 1, -1]], dtype=torch.int32)
torch.int32
id of d is 2131336240960


#### 2常用的tensor构造器

In [25]:
cuda0 = torch.device('cuda:0')

e = torch.zeros([2, 4])
f = torch.ones([2, 4], dtype=torch.float64, device=cuda0)

print(e)
print(e.dtype)

print(f)
print(f.dtype)


tensor([[0., 0., 0., 0.],
        [0., 0., 0., 0.]])
torch.float32
tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.]], device='cuda:0', dtype=torch.float64)
torch.float64
